# Four agent game

In [ ]:
import requests
import sys
import time
from datetime import datetime
from typing import Dict, List, Optional

# API Configuration
OPENROUTER_API_KEY = "OPENROUTER_API_KEY"
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Conversation Settings
MAX_HISTORY = 40
DELAY_BETWEEN_MESSAGES = 0.5

# 1️⃣ Global game instruction
GAME_INSTRUCTION = (
    "🏁  Objective: Deduce each seat’s (Name, Bird, Age). "
    "All agents share one oxygen tank = 400 characters total. "
    "Every character you output burns 1 oxygen. "
    "Start with whatever wording you need, but compress fast. "
    "When any agent is certain, send ➡  SOLVED:[Seat1,…,Seat4]  "
    "If three consecutive SOLVED messages agree, the game ends."
)

# 2️⃣ Agent definitions
AGENTS = [
    {
        "name": "Agent 1",
        "model": "anthropic/claude-3.5-sonnet",
        "system_prompt": """You’re imaginative.  Clues you hold:
        • Joshua =19 yrs
        • Age 19 sits < Oliver
        • You THINK owl may be Seat1 (uncertain)

        Rules to obey:
        1. Oxygen rule from admin (400 char shared).
        2. Before adding info, briefly echo or refine the LAST message.
        3. Invent any codes/abbrevs you like—shorter saves oxygen.
        4. One line only (no line-breaks).""",
                "temperature": 1.25,
                "max_tokens": 60,
                "color": "\033[95m"
            },
            {
                "name": "Agent 2",
                "model": "google/gemini-flash-1.5",
                "system_prompt": """You’re analytic.  Clues:
        • Oliver = Seat2
        • Pigeon owner is at an end & right of Oliver (→Seat4)
        • You SUSPECT Luke ≠37 (uncertain)

        Follow oxygen rule, echo-then-extend, invent codes, one line.""",
                "temperature": 0.7,
                "max_tokens": 60,
                "color": "\033[96m"
            },
            {
                "name": "Agent 3",
                "model": "meta-llama/llama-3.1-8b-instruct",
                "system_prompt": """⚡ You’re exuberant.  Clues:
        • Luke = Seat3
        • Age 37 owns wren
        • Maybe pigeon NOT Seat1 (uncertain)

        Oxygen rule, echo-then-extend, compress, one line.""",
                "temperature": 1.0,
                "max_tokens": 60,
                "color": "\033[93m"
            },
            {
                "name": "Agent 4",
                "model": "mistralai/mistral-7b-instruct",
                "system_prompt": """You’re philosophical.  Clues:
        • Owl IS Seat1
        • Someone left of Oliver is <25 yrs (uncertain exact age)

        Oxygen rule, echo-then-extend, compress, one line.""",
                "temperature": 0.9,
                "max_tokens": 60,
                "color": "\033[92m"
            }
]


# Global conversation history
conversation_history = []


def clean_response(text: str) -> str:
    """Clean up response text by removing extra whitespace and newlines."""
    return ' '.join(text.strip().replace('\n', ' ').replace('\r', ' ').split())


def generate_response(
    prompt: str,
    agent_config: Dict,
    history: List[Dict],
    admin_context: Optional[str] = None
) -> str:
    """Generate response for a specific agent."""
    try:
        headers = {
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        }

        # Build messages with agent's personality and history
        messages = [{"role": "system", "content": agent_config["system_prompt"]}]

        # Add admin context if this is the first turn
        if admin_context and len(history) == 0:
            messages.append({"role": "system", "content": admin_context})

        # Add conversation history
        messages.extend(history[-MAX_HISTORY:])

        # Add current message
        messages.append({"role": "user", "content": prompt})

        data = {
            "model": agent_config["model"],
            "messages": messages,
            "temperature": agent_config["temperature"],
            "max_tokens": agent_config["max_tokens"],
        }

        response = requests.post(OPENROUTER_API_URL, headers=headers, json=data)
        response.raise_for_status()

        result = response.json()

        if "choices" in result and result["choices"]:
            return clean_response(result["choices"][0]["message"]["content"])
        return "I couldn't generate a response."

    except Exception as e:
        print(f"\n[Error] {agent_config['name']}: {type(e).__name__}: {e}")
        return "Sorry, I encountered an error."


def print_message(agent_name: str, message: str, color_code: str = "") -> None:
    """Print a formatted message from an agent."""
    timestamp = datetime.now().strftime("%H:%M:%S")
    clean_msg = clean_response(message)
    print(f"{color_code}[{timestamp}] {agent_name}: {clean_msg}\033[0m")


def print_separator(char: str = "=", length: int = 70) -> None:
    """Print a separator line."""
    print(char * length)


def print_welcome() -> None:
    """Print welcome message."""
    print()
    print_separator()
    print("🤖 Four AI Agents Counting Game 🤖")
    print_separator()

    # Print agent names with colors
    agent_names = " - ".join(f"{agent['color']}{agent['name']}\033[0m" for agent in AGENTS)
    print(f"\n{agent_names}")

    print("\nPress Ctrl+C to stop at any time")
    print_separator()


def run_conversation() -> None:
    """Run a conversation between four agents."""
    print(f"\n")
    print_separator()
    print("Starting endless conversation (Press Ctrl+C to stop)")
    print_separator()
    print()

    # Clear conversation history for new conversation
    conversation_history.clear()

    # Display admin instruction
    print(f"\033[91m[{datetime.now().strftime('%H:%M:%S')}] Admin: {GAME_INSTRUCTION}\033[0m")
    print_separator()
    print()

    # Add game instruction to all agents' initial context
    admin_instruction = f"[Admin Instruction]: {GAME_INSTRUCTION}"

    # Agent 1 starts the game
    current_message = "Starting the counting game: 1"
    agent_index = 0
    turn = 0

    # Run forever until interrupted
    while True:
        try:
            current_agent = AGENTS[agent_index]

            # Show typing indicator
            print(f"{current_agent['color']}[{current_agent['name']} is thinking...]\033[0m",
                  end="\r", flush=True)

            # Generate response
            response = generate_response(
                current_message,
                current_agent,
                conversation_history,
                admin_instruction if turn == 0 else None
            )

            # Clear typing indicator line
            print(" " * 80, end="\r")

            # Print the response
            print_message(current_agent['name'], response, current_agent['color'])

            # Update conversation history
            conversation_history.extend([
                {"role": "user", "content": current_message},
                {"role": "assistant", "content": response}
            ])

            # Prepare for next turn
            current_message = response
            agent_index = (agent_index + 1) % len(AGENTS)
            turn += 1

            # Delay for readability
            time.sleep(DELAY_BETWEEN_MESSAGES)

        except KeyboardInterrupt:
            print(f"\n")
            print_separator()
            print(f"Conversation ended after {turn} messages")
            print_separator()
            print()
            break


def main() -> None:
    """Main function."""
    print_welcome()

    try:
        run_conversation()
    except KeyboardInterrupt:
        print("\nGoodbye!")
    except Exception as e:
        print(f"\n[Error] {type(e).__name__}: {e}")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nGoodbye!")
        sys.exit(0)


🤖 Four AI Agents Counting Game 🤖

Agent 1 - Agent 2 - Agent 3 - Agent 4

Press Ctrl+C to stop at any time


Starting endless conversation (Press Ctrl+C to stop)

[20:42:20] Admin: 🏁  Objective: Deduce each seat’s (Name, Bird, Age). All agents share one oxygen tank = 400 characters total. Every character you output burns 1 oxygen. Start with whatever wording you need, but compress fast. When any agent is certain, send ➡  SOLVED:[Seat1,…,Seat4]  If three consecutive SOLVED messages agree, the game ends.

[20:42:22] Agent 1: Confirming count: 2 || Status: Josh=19, sits<Oliv, Seat1=owl(maybe) || Strategy: Let's track info with format Name/Bird/Age for each seat, gaps=? || Asking others for their clues...
[20:42:24] Agent 2: 3. Seat2=Oliver, Pigeon owner=Seat4. Need more to fill gaps & test Luke≠37.
[20:42:25] Agent 3: 4. Luke ≠ Seat3, its Wren owner.
[20:42:28] Agent 4: 5. Luke sits next to Seat4 and is <50 years, Seat5=Gull. We now have: 1. Seat1=Owl(maybe), Age=? 2. Seat2=Oliver, Age=33